TASK1

In [65]:
import pandas as pd
import numpy as np
import random as random

scores = pd.read_csv('./data_safe_copy.csv')

print(f'Confirming that row count is at least 300: {len(scores) >= 300}\n')

print(scores.info())

print(f'\nScore is numeric: {scores['score'].apply(lambda x: isinstance(x, (int, float))).all()}') 

Confirming that row count is at least 300: True

<class 'pandas.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   student_id  300 non-null    str  
 1   cohort      300 non-null    str  
 2   module      300 non-null    str  
 3   assignment  300 non-null    str  
 4   score       300 non-null    int64
dtypes: int64(1), str(4)
memory usage: 11.8 KB
None

Score is numeric: True


TASK2

In [67]:




existing_student_ids = set(scores['student_id'])
new_ids = []
counter = 1

while len(new_ids) < 10:
    new_id = f'S{counter:03d}'
    if new_id not in existing_student_ids:
        new_ids.append(new_id)
    counter += 1

statuses = [random.choice(['active', 'inactive']) for _ in new_ids]

roster = pd.DataFrame({
    'student_id': new_ids,
    'status': statuses
})


merged = pd.merge(scores, roster, on='student_id', how='left')

print(merged)

print(f'''
Confirming that missing statuses still exist after merge: {all(merged['status'].isna())}
Missing statuses count: {sum(merged['status'].isna())}
''')

    student_id cohort   module assignment  score status
0         S001  alpha  Module1         A1     78    NaN
1         S001  alpha  Module1         A2     84    NaN
2         S001  alpha  Module2         A1     79    NaN
3         S001  alpha  Module2         A2     86    NaN
4         S001  alpha  Module3         A1     81    NaN
..         ...    ...      ...        ...    ...    ...
295       S050   beta  Module1         A2     77    NaN
296       S050   beta  Module2         A1     73    NaN
297       S050   beta  Module2         A2     76    NaN
298       S050   beta  Module3         A1     75    NaN
299       S050   beta  Module3         A2     78    NaN

[300 rows x 6 columns]

Confirming that missing statuses still exist after merge: True
Missing statuses count: 300



TASK3

In [80]:

cohort_groupby = scores.groupby(scores['cohort'])
module_groupby = scores.groupby(scores['module'])

average_cohort_score = cohort_groupby['score'].mean()
average_module_score = module_groupby['score'].mean()


avg_by_module = pd.DataFrame({
    'Average score by cohort': average_cohort_score,
    'Average score by module': average_module_score
}) 

cohort_mean = average_cohort_score.mean()
module_mean = average_module_score.mean()

print(f'Module-level averages are higher than cohort level: {module_mean-cohort_mean>0}')


Module-level averages are higher than cohort level: True


TASK4

In [81]:
student_module_report = scores.pivot_table(
    index='student_id',
    columns='module',
    values='score',
    aggfunc='mean'
)

student_module_report

module,Module1,Module2,Module3
student_id,,,
S001,81.0,82.5,84.5
S002,73.5,72.0,75.0
S003,87.5,88.5,89.5
S004,68.5,67.5,69.5
S005,81.5,82.5,83.5
S006,76.0,75.0,77.0
S007,89.0,89.0,90.5
S008,71.0,70.0,72.0
S009,79.5,78.5,80.5


TASK5

In [102]:
student_avg = scores.groupby(['student_id', 'cohort'], as_index=False)['score'].mean()
student_avg.rename(columns={'score': 'avg_score'}, inplace=True)

student_avg['rank'] = student_avg.groupby('cohort')['avg_score'].rank(method='first', ascending=False)

top_students = student_avg[student_avg['rank'] <= 3].drop(columns='rank')


top_students = pd.DataFrame({
    'student_id': top_students['student_id'],
    'cohort': top_students['cohort'],
    'avg_score': top_students['avg_score']
})

top_students = top_students.sort_values(
    by=['cohort', 'avg_score'], 
    ascending=[True, False]
).reset_index(drop=True)

cohort_counts = top_students['cohort'].value_counts()
print(f'Validating that all cohorts appear exatly 3 times: {all(cohort_counts == 3)}')

Validating that all cohorts appear exatly 3 times: True
